In [1]:
#imports
import numpy as np
import torch.optim as optim
import math
import argparse
import sys

from torch import nn
from dataloader import *
from models import *
from helper import *
from trainAndtest import *

In [2]:
percentage = 0.05

X_train, X_val, train_nc, X_test = DataProcessing(float(percentage))
train_loader = DataLoader(X_train, batch_size=16, shuffle=True, num_workers=16)
val_loader = DataLoader(X_val, batch_size=16, shuffle=True, num_workers=16)
test_loader = DataLoader(X_test, batch_size=16, shuffle=True, num_workers=16)

=============== Loading Data ===============


In [3]:
## 
mname = "again2_train.pth"


Getting resnet50
pretrained
================ Model Created ===============


In [4]:
    model_type = "resnet50"

    model = get_from_models(model_type, 17, not(mname==None), mname)
    model = model.cuda()

    print("================ Model Created ===============")


    # Created the loss function and optimizer
    loss_fn = nn.MultiLabelSoftMarginLoss().cuda()
    optimizer = optim.SGD([
        {'params': [param for name, param in model.named_parameters() if name[-4:] == 'bias']},
        {'params': [param for name, param in model.named_parameters() if name[-4:] != 'bias'],
        'weight_decay': 1e-4}
    ], lr=1e-2, momentum=0.9, nesterov=True)


In [5]:
load_best = "bestval.txt"

best_score, best_threshold = saveCreds(not(load_best == None))
best_model = model

In [ ]:
for epoch in range(1, 5):
    train(model, epoch, train_loader, optimizer, loss_fn)
    torch.save(model.state_dict(),"./{}_train.pth".format(23))
    v_score, threshold = validate(model, val_loader, loss_fn)

    if v_score < best_score:
        best_model = model
        best_threshold = threshold
        best_score = v_score
        saveCreds(save = True, threshold = best_threshold, v_score = best_score)
        torch.save(model.state_dict(),"./{}_train.pth".format("best"))

#     X_train.set_transformation()
#     train_loader = DataLoader(X_train, batch_size=16, shuffle=True, num_workers=16)


Train Epoch: 1 [0/36432 (0%)]	Loss: 0.634587
Train Epoch: 1 [160/36432 (0%)]	Loss: 0.270923
Train Epoch: 1 [320/36432 (1%)]	Loss: 0.244371
Train Epoch: 1 [480/36432 (1%)]	Loss: 0.168792
Train Epoch: 1 [640/36432 (2%)]	Loss: 0.261826
Train Epoch: 1 [800/36432 (2%)]	Loss: 0.146568
Train Epoch: 1 [960/36432 (3%)]	Loss: 0.221054
Train Epoch: 1 [1120/36432 (3%)]	Loss: 0.146117
Train Epoch: 1 [1280/36432 (4%)]	Loss: 0.126556
Train Epoch: 1 [1440/36432 (4%)]	Loss: 0.162494
Train Epoch: 1 [1600/36432 (4%)]	Loss: 0.202241
Train Epoch: 1 [1760/36432 (5%)]	Loss: 0.135179
Train Epoch: 1 [1920/36432 (5%)]	Loss: 0.172321
Train Epoch: 1 [2080/36432 (6%)]	Loss: 0.131704
Train Epoch: 1 [2240/36432 (6%)]	Loss: 0.190070
Train Epoch: 1 [2400/36432 (7%)]	Loss: 0.134154
Train Epoch: 1 [2560/36432 (7%)]	Loss: 0.149876
Train Epoch: 1 [2720/36432 (7%)]	Loss: 0.125052
Train Epoch: 1 [2880/36432 (8%)]	Loss: 0.155880
Train Epoch: 1 [3040/36432 (8%)]	Loss: 0.120475
Train Epoch: 1 [3200/36432 (9%)]	Loss: 0.143644
T

Train Epoch: 3 [8000/36432 (22%)]	Loss: 0.120176
Starting Validation
Done 0.0%
Done 7.90513833992%
Done 15.8102766798%
Done 23.7154150198%
Done 31.6205533597%
Done 39.5256916996%
Done 47.4308300395%
Done 55.3359683794%
Done 63.2411067194%
Done 71.1462450593%
Done 79.0513833992%
Done 86.9565217391%
Done 94.8616600791%
Calling get_optimal_threshhold


In [6]:
predict(test_loader, best_model, best_threshold, X_test, "Prediction")

====== Starting Prediction ======
Done 0.0%
Done 7.90904597133%
Done 15.8180919427%
Done 23.727137914%
Done 31.6361838853%
Done 39.5452298566%
Done 47.454275828%
Done 55.3633217993%
Done 63.2723677706%
Done 71.181413742%
Done 79.0904597133%
Done 86.9995056846%
Done 94.908551656%
====== Raw predictions done ========
======= Final predictions done =======
Final predictions saved to ./out/Prediction-final-pred-.csv
Final fbeta score is 92.49620501214832
